In [58]:
# import module libraries
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import pandas as pd
import optuna
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [59]:
# LSTM with layer normalization
class LayerNormLSTM(nn.Module):
    def __init__(self, input_node, hidden_node):
        super().__init__()
        self.input_node = input_node
        self.hidden_node = hidden_node

        self.w_i = nn.Linear(input_node, 4*hidden_node, bias=False)
        self.w_h = nn.Linear(hidden_node, 4*hidden_node, bias=False)

        self.ln_i = nn.LayerNorm(hidden_node)
        self.ln_f = nn.LayerNorm(hidden_node)
        self.ln_w = nn.LayerNorm(hidden_node)
        self.ln_o = nn.LayerNorm(hidden_node)
        self.ln_c = nn.LayerNorm(hidden_node)

    def forward(self, input, hidden):
        h_prev, c_prev = hidden

        gi = self.w_i(input)
        gh = self.w_h(h_prev)
        i_i, i_f, i_w, i_o = gi.chunk(4, dim=-1)
        h_i, h_f, h_w, h_o = gh.chunk(4, dim=-1)

        i_g = torch.sigmoid(self.ln_i(i_i + h_i))
        f_g = torch.sigmoid(self.ln_f(i_f + h_f))
        w_g = torch.tanh(self.ln_w(i_w + h_w))
        o_g = torch.sigmoid(self.ln_o(i_o + h_o))
        

        c_new = f_g * c_prev + i_g * w_g
        c_new = self.ln_c(c_new)

        h_new = o_g * torch.tanh(c_new)

        return h_new, c_new

In [60]:
# State feature extractor using LayerNorm LSTM
class StateExtr(nn.Module):
    def __init__(self, input_node, hidden_node, n_layer, dropout):
        super().__init__()
        self.hidden_node = hidden_node
        self.n_layer = n_layer
        self.input_node = input_node

        self.lstm_cells = nn.ModuleList()
        self.lstm_cells.append(LayerNormLSTM(input_node, hidden_node))

        for i in range(n_layer - 1):
            self.lstm_cells.append(LayerNormLSTM(hidden_node, hidden_node))

        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(hidden_node)

    def forward(self, x, seq_len):
        batch_size, max_len, _ = x.size()
        device = x.device

        h_states = []
        c_states = []

        for _ in range(self.n_layer):
            h_states.append(torch.zeros(batch_size, self.hidden_node, device=device))
            c_states.append(torch.zeros(batch_size, self.hidden_node, device=device))

        outputs = []
        for t in range(max_len):
            x_t = x[:, t, :]

            layer_input = x_t # initialize layer input with input tensor
            for layer_idx, lstm_cell in enumerate(self.lstm_cells):
                h_new, c_new = lstm_cell(layer_input, (h_states[layer_idx], c_states[layer_idx]))
                
                h_states[layer_idx] = h_new
                c_states[layer_idx] = c_new

                if layer_idx < len(self.lstm_cells) - 1:
                    layer_input = self.dropout(h_new)
                else:
                    layer_input = h_new

            outputs.append(layer_input)

        output_tensor = torch.stack(outputs, dim=1)
        seq_len_cpu = seq_len.detach().cpu().long()
        mask = torch.arange(max_len, device='cpu')[None, :] < seq_len_cpu[:, None]
        mask = mask.float().to(device).unsqueeze(-1)

        masked_output = output_tensor * mask
        normed_output = self.layernorm(masked_output)
        return self.dropout(normed_output)

In [61]:
# Physical change regressor
class PhysRegr(nn.Module):
    def __init__(self, input_node, output_node, n_layer, hidden_node, dropout):
        super().__init__()

        layers = []

        layers.extend([
            nn.Linear(input_node, hidden_node),
            nn.LayerNorm(hidden_node),
            nn.ReLU(),
            nn.Dropout(dropout)
        ])

        for _ in range(n_layer - 1):
            layers.extend([
                nn.Linear(hidden_node, hidden_node),
                nn.LayerNorm(hidden_node),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
        
        layers.append(nn.Linear(hidden_node, output_node))

        self.layers = nn.Sequential(*layers)

    def forward(self, hidden_states):
        return self.layers(hidden_states)

In [62]:
# Current regressor
class CurrRegr(nn.Module):
    def __init__(self, input_node, hidden_node, n_layer, dropout):
        super().__init__()

        layers = []

        layers.extend([
            nn.Linear(input_node, hidden_node),
            nn.LayerNorm(hidden_node),
            nn.ReLU(),
            nn.Dropout(dropout)
        ])

        for _ in range(n_layer - 1):
            layers.extend([
                nn.Linear(hidden_node, hidden_node),
                nn.LayerNorm(hidden_node),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
        
        layers.append(nn.Linear(hidden_node, 1))

        self.layers = nn.Sequential(*layers)

    def forward(self, hidden_states):
        return self.layers(hidden_states)   

In [63]:
# Physical Constraint Layer
class PhysConstr(nn.Module):
    def __init__(self, range_mm, curr_regr, eps=1e-2):
        super().__init__()
        
        self.eps = eps
        self.curr_regr = curr_regr
        self.register_buffer('range_mm_tensor',self._range2tensor(range_mm))

    def _range2tensor(self, range_mm):
        feature_names = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK', 'CBK', 'I']
        ranges = torch.zeros(len(feature_names), 2)

        for i, name in enumerate(feature_names):
            ranges[i, 0] = range_mm[name]['min']
            ranges[i, 1] = range_mm[name]['max']

        return ranges

    def _norm_tensor(self, data, feature_idx):
        min_val = self.range_mm_tensor[feature_idx, 0]
        max_val = self.range_mm_tensor[feature_idx, 1]
        return (data - min_val) / (max_val - min_val)

    def _denorm_tensor(self, norm_data, feature_idx):
        min_val = self.range_mm_tensor[feature_idx, 0]
        max_val = self.range_mm_tensor[feature_idx, 1]
        return norm_data * (max_val - min_val) + min_val

    def forward(self, phys_chng, cur_state, fin, initV):
        V_idx, E_idx, VF_idx, VA_idx, VB_idx = 0, 1, 2, 3, 4
        CFLA_idx, CALA_idx, CFK_idx, CBK_idx, I_idx = 5, 6, 7, 8, 9

        VF = self._denorm_tensor(cur_state[..., 2:3], VF_idx)
        VA = self._denorm_tensor(cur_state[..., 3:4], VA_idx)
        VB = self._denorm_tensor(cur_state[..., 4:5], VB_idx)
        CFLA = self._denorm_tensor(cur_state[..., 5:6], CFLA_idx)
        CALA = self._denorm_tensor(cur_state[..., 6:7], CALA_idx)
        CFK = self._denorm_tensor(cur_state[..., 7:8], CFK_idx)
        CBK = self._denorm_tensor(cur_state[..., 8:9], CBK_idx)

        ## Flow in parameters
        FvF, FvA, FvB, CiLA, CiK = fin

        VFi, VAi, VBi = initV

        dVF_in, dVA_in, dVB_in = FvF, FvA, FvB
        dNFLA_in, dNFK_in = FvF * CiLA, FvF * CiK

        dVA = phys_chng[..., 0:1]
        dVB = phys_chng[..., 1:2]
        rratio = phys_chng[..., 2:3]
        dNBK = phys_chng[..., 3:4]

        ## physical boundary: dNALA <= dNBK
        ratio = torch.sigmoid(rratio)
        dNALA = ratio * dNBK

        ## Mass Balance
        NFLA = CFLA * VF
        NALA = CALA * VA
        NFK = CFK * VF
        NBK = CBK * VB

        ### conservation
        dVA = torch.where(VF < dVA + dVB, VF * dVA/(dVA+dVB), dVA)
        dVB = torch.where(VF < dVA + dVB, VF * dVB/(dVA+dVB), dVB)
        dNALA = torch.where(NFLA < dNALA, NFLA, dNALA)
        dNBK = torch.where(NFK < dNBK, NBK, dNBK)

        ### new states before discharge
        nVF_bf = VF - dVA - dVB + dVF_in
        nVA_bf = VA + dVA + dVA_in
        nVB_bf = VB + dVB + dVB_in

        nNFLA_bf = NFLA - dNALA + dNFLA_in
        nNALA_bf = NALA + dNALA
        nNFK_bf = NFK - dNBK + dNFK_in
        nNBK_bf = NBK + dNBK

        nCFLA = nNFLA_bf / nVF_bf
        nCALA = nNALA_bf / nVA_bf
        nCFK = nNFK_bf / nVF_bf
        nCBK = nNBK_bf / nVB_bf

        dVF_out = torch.clamp(nVF_bf - VFi, min=0.0)
        dVA_out = torch.clamp(nVA_bf - VAi, min=0.0)
        dVB_out = torch.clamp(nVB_bf - VBi, min=0.0)

        nVF = nVF_bf - dVF_out
        nVA = nVA_bf - dVA_out
        nVB = nVB_bf - dVB_out
        
        dNFLA_out, dNFK_out = nCFLA*dVF_out, nCFK*dVF_out

        dNALA_out = nCALA*dVA_out
        dNBK_out = nCBK*dVB_out

        ### Final new states
        nNFLA = nNFLA_bf - dNFLA_out
        nNALA = nNALA_bf - dNALA_out
        nNFK = nNFK_bf - dNFK_out
        nNBK = nNBK_bf - dNBK_out

        V = cur_state[..., 0:1]
        E = cur_state[..., 1:2]

        nVF_norm = self._norm_tensor(nVF, VF_idx)
        nVA_norm = self._norm_tensor(nVA, VA_idx)
        nVB_norm = self._norm_tensor(nVB, VB_idx)
        nCFLA_norm = self._norm_tensor(nCFLA, CFLA_idx)
        nCALA_norm = self._norm_tensor(nCALA, CALA_idx)
        nCFK_norm = self._norm_tensor(nCFK, CFK_idx)
        nCBK_norm = self._norm_tensor(nCBK, CBK_idx)

        temp_state = torch.cat([
            V, E, nVF_norm, nVA_norm, nVB_norm, nCFLA_norm, nCALA_norm, nCFK_norm, nCBK_norm
        ], dim=-1)

        nI_pred = self.curr_regr(temp_state)
        nI_real = self._denorm_tensor(nI_pred, I_idx)
        nI_real = torch.clamp(nI_real, min=0.0)
        nI_norm = self._norm_tensor(nI_real, I_idx)

        next_state = torch.cat([
            V, E, nVF_norm, nVA_norm, nVB_norm, nCFLA_norm, nCALA_norm, nCFK_norm, nCBK_norm, nI_norm
        ], dim=-1)

        ### discharge
        discharge = {
            'VF': dVF_out,
            'VA': dVA_out,
            'VB': dVB_out,
            'NFLA': dNFLA_out,
            'NALA': dNALA_out,
            'NFK': dNFK_out,
            'NBK': dNBK_out,
            'CFLA': nCFLA,
            'CALA': nCALA,
            'CFK': nCFK,
            'CBK': nCBK
        }

        return next_state, discharge

In [64]:
# BMED model
class BMEDModel(nn.Module):
    def __init__(self, state_extr_params, phys_regr_params, curr_regr_params, range_mm):
        super().__init__()
        self.state_extr = StateExtr(**state_extr_params)
        self.phys_regr = PhysRegr(**phys_regr_params)
        self.curr_regr = CurrRegr(**curr_regr_params)
        self.phys_constr = PhysConstr(range_mm, self.curr_regr)

        self._hidden_states = None
        self._cell_states = None

    def _reset_hidden_states(self, batch_size, device):
        self._hidden_states = []
        self._cell_states = []
        for _ in range(self.state_extr.n_layer):
            self._hidden_states.append(torch.zeros(batch_size, self.state_extr.hidden_node, device=device))
            self._cell_states.append(torch.zeros(batch_size, self.state_extr.hidden_node, device=device))

    def cont_sim(self, init_state, target_len, fin, initV):
        batch_size = init_state.size(0)
        feature_size = init_state.size(1)
        device = init_state.device

        self._reset_hidden_states(batch_size, device)

        pred = torch.zeros(batch_size, target_len, feature_size, device=device)
        discharge_record = []
        cur_state = init_state.clone()

        for t in range(target_len):
            pred[:, t, :] = cur_state

            if t < target_len - 1:
                lstm_input = cur_state[:, :-1] # except current
                hidden_output = self._lstm_single_step(lstm_input)

                phys_chng = self.phys_regr(hidden_output.unsqueeze(1))
                cur_state_expanded = cur_state.unsqueeze(1)

                next_state, discharge = self.phys_constr(
                    phys_chng, cur_state_expanded, fin, initV
                )

                cur_state = next_state.squeeze(1)
                discharge_record.append(discharge)
        return pred, discharge_record

    def _lstm_single_step(self, x_t):
        layer_input = x_t

        for layer_idx, lstm_cell in enumerate(self.state_extr.lstm_cells):
            h_new, c_new = lstm_cell(layer_input, (self._hidden_states[layer_idx], self._cell_states[layer_idx]))
            
            self._hidden_states[layer_idx] = h_new
            self._cell_states[layer_idx] = c_new

            if layer_idx < len(self.state_extr.lstm_cells) - 1:
                layer_input = self.state_extr.dropout(h_new)
            else:
                layer_input = h_new # last layer output

        normed_output = self.state_extr.layernorm(layer_input)
        return self.state_extr.dropout(normed_output)

    def forward(self, init_state, target_len, fin, initV):
        return self.cont_sim(init_state, target_len, fin, initV)

In [65]:
# Utility functions

## Normalize input data with the min-max normalization range of pre-trained model
def normalize(inputs, range_mm):
    features = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK', 'CBK']
    norm = []

    for _, (name, value) in enumerate(zip(features, inputs)):
        min_val = range_mm[name]['min']
        max_val = range_mm[name]['max']
        norm_val = (value - min_val) / (max_val - min_val)
        norm.append(norm_val)
    
    return norm

def denormalize(outputs, range_mm):
    """출력값들을 실제 물리적 값으로 변환"""
    feature_names = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK', 'CBK', 'I']
    denormalized = np.zeros_like(outputs)
    
    for i, name in enumerate(feature_names):
        if name in range_mm:
            min_val = range_mm[name]['min']
            max_val = range_mm[name]['max']
            denormalized[:, :, i] = outputs[:, :, i] * (max_val - min_val) + min_val
        else:
            denormalized[:, :, i] = outputs[:, :, i]
    
    return denormalized

In [66]:
# Load trained model
model_path = 'BMED_FR_250916.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Model: {model_path}')
print(f'Device: {device}')

model = torch.load(model_path, map_location=device, weights_only=False)
model_config = model['model_config']
state_extr_params = model_config['state_extr_params']
phys_regr_params = model_config['phys_regr_params']
curr_regr_params = model_config['curr_regr_params']
model_range_mm = model_config['range_mm']

simulator = BMEDModel(
    state_extr_params = state_extr_params,
    phys_regr_params = phys_regr_params,
    curr_regr_params = curr_regr_params,
    range_mm = model_range_mm
).to(device)

simulator.load_state_dict(model['model_state_dict'])
simulator.eval()

print('Load model parameters')

Model: BMED_FR_250916.pth
Device: cuda
Load model parameters


In [67]:
def objective(trial):
    # maximize LA productivity
    # experimental conditions
    V = trial.suggest_float('Voltage',10, 35, step=2.5)
    E = trial.suggest_float('Electrolyte',0.25, 1.0,step=0.05)
    CFLA = trial.suggest_float('LA conc', 0.5, 3.0, step=0.5)
    # cQF = trial.suggest_float('Feed Flow rate', 10,20, step=0.5)
    # cQA = trial.suggest_float('Acid Flow rate', 10,20, step=0.5)
    # cQB = trial.suggest_float('Base Flow rate', 10,20, step=0.5)

    cond_init = [V, E, 0.7, 0.7, 0.7, CFLA, 0, CFLA*2, 0]
    simulation_time = 400

    # QF, QA, QB = cQF, cQA, cQB
    QF, QA, QB = 10, 10, 10
    cond_flow = [QF*60/1000*0.25, QA*60/1000*0.25, QB*60/1000*0.25, CFLA, CFLA*2]

    initV = [0.7, 0.7, 0.7]
    norm_inputs = normalize(cond_init, model_range_mm)
    
    init_normI = 0.0
    init_state_values = norm_inputs + [init_normI]
    init_state_tensor = torch.tensor([init_state_values]).float().to(device)

    with torch.no_grad():
        pred, discharge_record = simulator(init_state_tensor, simulation_time, cond_flow, initV)

    pred_norm = pred.cpu().numpy()
    pred_real = denormalize(pred_norm, model_range_mm)

    time_steps = np.arange(simulation_time)*0.25

    sim_res = {
        'time': time_steps,
        'V': pred_real[0, :, 0],
        'E': pred_real[0, :, 1], 
        'VF': pred_real[0, :, 2],
        'VA': pred_real[0, :, 3],
        'VB': pred_real[0, :, 4],
        'CFLA': pred_real[0, :, 5],
        'CALA': pred_real[0, :, 6],
        'CFK': pred_real[0, :, 7],
        'CBK': pred_real[0, :, 8],
        'I': pred_real[0, :, 9]
    }
    
    discharge_data = {
        'time': time_steps[:-1],  # discharge는 simulation_time-1 개
        'VF_out': [],
        'VA_out': [],
        'VB_out': [],
        'CFLA_out': [],
        'CALA_out': [],
        'CFK_out': [],
        'CBK_out': [],
        'NFLA_out': [],
        'NALA_out': [],
        'NFK_out': [],
        'NBK_out': []
    }

    ## Extract discharge results from the record variable
    for discharge in discharge_record:
        discharge_data['VF_out'].append(discharge['VF'].item()/0.25)
        discharge_data['VA_out'].append(discharge['VA'].item()/0.25)
        discharge_data['VB_out'].append(discharge['VB'].item()/0.25)
        discharge_data['CFLA_out'].append(discharge['CFLA'].item())
        discharge_data['CALA_out'].append(discharge['CALA'].item())
        discharge_data['CFK_out'].append(discharge['CFK'].item())
        discharge_data['CBK_out'].append(discharge['CBK'].item())
        discharge_data['NFLA_out'].append(discharge['NFLA'].item()/0.25)
        discharge_data['NALA_out'].append(discharge['NALA'].item()/0.25)
        discharge_data['NFK_out'].append(discharge['NFK'].item()/0.25)
        discharge_data['NBK_out'].append(discharge['NBK'].item()/0.25)

    input_LA_rate = cond_flow[0]/0.25 * cond_flow[3]  # mol/hr
    recovery_efficiency = np.array(discharge_data['NALA_out']) / input_LA_rate * 100
    #Performance calculation
    I_st = sim_res['I'][-50] # A = C/s
    C_st = 3600*I_st # C/hr
    CE_st = np.array(discharge_data['NALA_out'][-50])*96485/10/C_st*100 # %
    Fm_st = np.array(discharge_data['NALA_out'][-50])*90.1 # g/hr
    SEC_st = cond_init[0]*I_st/Fm_st # Wh/g = kWh/kg
    t_st = 1000/Fm_st # hr/kg
    E_pump = 0.0047*t_st # kWh/kg
    VA_st = np.array(discharge_data['VA_out'][-50]) # L/hr = kg/hr
    E_dist = (VA_st *4.184*(100-25)*t_st + (VA_st - Fm_st/0.85/1000)*43.9*1000/18)/3600 # kWh/kg
    RR_st = recovery_efficiency[-50]
    TotalE_st = SEC_st + E_pump + E_dist
    

    return RR_st, Fm_st, TotalE_st

In [68]:

db_url = f"sqlite:///bmed_1stage_optimization_LA productivity.db"

study = optuna.create_study(
    directions= ['maximize', 'maximize', 'minimize'],
    study_name='bmed_1stage_optimization_LA productivity',
    sampler=optuna.samplers.TPESampler(),
    storage=db_url,
    load_if_exists=True
)

study.optimize(objective, n_trials=100)



[I 2025-09-29 16:46:24,228] A new study created in RDB with name: bmed_1stage_optimization_LA productivity
[I 2025-09-29 16:46:25,986] Trial 0 finished with values: [30.379603306452434, 49.26964064240455, 2.3817738734011606] and parameters: {'Voltage': 25.0, 'Electrolyte': 0.7, 'LA conc': 3.0}.
[I 2025-09-29 16:46:27,719] Trial 1 finished with values: [49.85346893469493, 26.950785306096076, 3.1252737909033006] and parameters: {'Voltage': 12.5, 'Electrolyte': 0.3, 'LA conc': 1.0}.
[I 2025-09-29 16:46:29,483] Trial 2 finished with values: [86.54139439264934, 46.784277808666225, 3.732220593294868] and parameters: {'Voltage': 35.0, 'Electrolyte': 0.75, 'LA conc': 1.0}.
[I 2025-09-29 16:46:31,224] Trial 3 finished with values: [36.455317338307694, 49.26936138272285, 3.492609323814992] and parameters: {'Voltage': 35.0, 'Electrolyte': 0.55, 'LA conc': 2.5}.
[W 2025-09-29 16:46:32,966] Trial 4 failed with parameters: {'Voltage': 35.0, 'Electrolyte': 0.9500000000000001, 'LA conc': 0.5} because 

In [69]:
def analyze_pareto_front(study):
    """Pareto front 분석 및 시각화 및 best 조건 출력"""
    trials = study.trials

    # 목적함수 값 추출 및 best trial 찾기
    objectives = []
    best_idx = None
    best_score = None

    for idx, trial in enumerate(trials):
        if trial.state == optuna.trial.TrialState.COMPLETE:
            objectives.append(trial.values)
            # best 조건: Recovery Ratio, Flux 최대, Energy Efficiency 최소
            # 간단히 weighted sum으로 best trial 선정 (가중치는 임의, 필요시 조정)
            score = trial.values[0] + trial.values[1] - trial.values[2]
            if (best_score is None) or (score > best_score):
                best_score = score
                best_idx = idx

    objectives = np.array(objectives)

    # 3D scatter plot으로 Pareto front 시각화
    fig = go.Figure(data=go.Scatter3d(
        x=objectives[:, 0],  # RR_st
        y=objectives[:, 1],  # Fm_st 
        z=objectives[:, 2],  # TotalE_st
        mode='markers',
        marker=dict(
            size=5,
            color=objectives[:, 0],  # Recovery Ratio로 색상 구분
            colorscale='Viridis',
            showscale=True
        )
    ))

    fig.update_layout(
        title='Pareto Front Analysis',
        scene=dict(
            xaxis_title='Recovery Ratio (%)',
            yaxis_title='Flux (g/hr)',
            zaxis_title='Energy Efficiency (kWh/kg)'
        )
    )

    # best 조건 출력
    if best_idx is not None:
        # best_idx는 전체 trials 기준이므로, COMPLETE trial만 추출한 objectives와 인덱스가 다를 수 있음
        # 따라서 best trial을 직접 찾음
        best_trial = None
        best_score = None
        for trial in trials:
            if trial.state == optuna.trial.TrialState.COMPLETE:
                score = trial.values[0] + trial.values[1] - trial.values[2]
                if (best_score is None) or (score > best_score):
                    best_score = score
                    best_trial = trial
        print("Best 조건 (가중합 기준):")
        print(f"  Recovery Ratio: {best_trial.values[0]:.2f} %")
        print(f"  Flux: {best_trial.values[1]:.2f} g/hr")
        print(f"  Energy Efficiency: {best_trial.values[2]:.2f} kWh/kg")
        print("  Parameters:", best_trial.params)

    return fig

# 결과 분석
fig = analyze_pareto_front(study)
fig.show()

Best 조건 (가중합 기준):
  Recovery Ratio: 86.54 %
  Flux: 46.78 g/hr
  Energy Efficiency: 3.73 kWh/kg
  Parameters: {'Voltage': 35.0, 'Electrolyte': 0.75, 'LA conc': 1.0}


AttributeError: module 'nbformat' has no attribute '__version__'

In [ ]:
import pandas as pd

# Optuna study에서 모든 trial의 결과를 DataFrame으로 변환
def save_optuna_trials_to_csv(study, filename="optuna_trials.csv"):
    # 각 trial의 값과 파라미터를 합쳐서 DataFrame 생성
    records = []
    for trial in study.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            record = {
                "Trial Number": trial.number,
                "Recovery Ratio": trial.values[0],
                "Flux": trial.values[1],
                "Energy Efficiency": trial.values[2],
            }
            # 파라미터 추가
            record.update(trial.params)
            records.append(record)
    df = pd.DataFrame(records)
    df.to_csv(filename, index=False)
    print(f"Optuna trial 결과가 '{filename}' 파일로 저장되었습니다.")

# 사용 예시
save_optuna_trials_to_csv(study, "bmed_1stage_optuna_trials.csv")


Optuna trial 결과가 'bmed_1stage_optuna_trials.csv' 파일로 저장되었습니다.
